In [1]:
%matplotlib notebook
import time
import sklearn
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVC
from collections import Counter
from numpy.linalg.linalg import LinAlgError
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import tree
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix as confu
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.cluster import KMeans

/usr/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# Normalize data
def normalization(training_numpy, testing_numpy):
    min_max = preprocessing.MinMaxScaler()
    min_max.fit(training_numpy)
    x_scaled = min_max.fit_transform(training_numpy)
    testing_x_scaled = min_max.transform(testing_numpy)
    #training_norm = pd.DataFrame(x_scaled, columns = columns)
    #testing_norm = pd.DataFrame(testting_x_scaled, columns = columns)
    return (x_scaled, testing_x_scaled)

def pca_function(rate, data):
    pca = PCA()
    pca.fit(data)
    for thres_n in xrange(1,len(data)):
        if sum(pca.explained_variance_ratio_[:thres_n])>rate:
            pca_n = thres_n
            break
    
    pca = PCA(n_components=pca_n)
    pca_data = pca.fit_transform(data)
    return (pca_data, pca)

def lda_function(X, y):
    n_cluster = 4
    #lda = LDA()
    #lda_data = lda.fit_transform(X, y)
    original_y = y.copy() 
    #normal_index = np.where(y == 0)[0]
    anomaly_index = np.where(y == 1)[0]
    #print len(normal_index), len(anomaly_index)
    
    #km_normal = KMeans(n_clusters=2)
    #km_normal.fit(X.iloc[normal_index], y[normal_index])
    km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly.fit(X.iloc[anomaly_index], y[anomaly_index])
    
   
    #normal_label = km_normal.labels_    
    #normal_label[np.where(normal_label == 0)[0]] = 0
    #normal_label[np.where(normal_label == 1)[0]] = 1
    
    anomaly_label = km_anomaly.labels_
    #print 'before label:',np.unique(anomaly_label)
    for i in range(len(np.unique(anomaly_label)), 0, -1):
        #print i, np.unique(anomaly_label)
        anomaly_label[np.where(anomaly_label == (i-1))[0]] = i
    #print 'label:',np.unique(anomaly_label)

    #y[normal_index] = normal_label
    y[anomaly_index] = anomaly_label
    
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


'''
def pca_function(n, data):
    pca = PCA(n_components=n)
    pca_data = pca.fit_transform(data)
    return (pca_data.flatten(), pca)

def lda_function(X, y):
    lda = LDA()
    lda_data = lda.fit_transform(X, y)
    return (lda_data.flatten(), lda)
'''
def replace_value(training_normal_numpy, key, loc):  
    training_normal_numpy[loc][key] = 1
    
def del_zero_column(df_train, df_test):
    df_train = df_train.loc[:, (df_train != 0).any(axis=0)]
    df_test = df_test[df_train.columns]
    return (df_train, df_test)
    

In [ ]:
key_str = {'Normal':0, 'Botnet':1, 'Background':2}
column_unuse = ['StartTime', 'SrcAddr', 'DstAddr', 'Sport']
column_str = ['Proto', 'State','Dir','Dport']
score_list = {}
fscore_list = {}
#output_fscore = pd.DataFrame()
output_fscore = pd.DataFrame(columns=['lda_2_class', '0.9_pca', '1_dim_pca'])
#output_fscore = pd.DataFrame(columns=['Numeric_proto', 'Numeric_state', 'Numeric_dir', 'Numeric_dport', 'Binary_proto', 'Binary_state', 'Binary_dir','Binary_dport'])

pca_rate = 0.9



   
train = pd.read_csv('2014-06-06_capture-win8.binetflow.labeled')

# Fill 0 to missing value
train = train.fillna(0)    
train = train[train['State'] != 0]

# Drop unuse columns and row with missing value
missing_state = []
map(lambda word: missing_state.append(word) if word[-1]=='_' else None, np.unique(train['State']))

for column in column_unuse:
    del train[column]

for state in missing_state:
    train = train[train['State'] != state]


# Transfer str label to int lable
label_num = []

map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])
train['Label'] = label_num





# training & testing index
train_index = np.array(np.where(train['Label'] == 0)[0].tolist() + np.where(train['Label'] == 1)[0].tolist())
print  'training length:', len(train_index)

now_lda_fscore = []

all_score_lda = []

skf = StratifiedKFold(np.array(label_num)[train_index], n_folds=10)
val_label = np.array(label_num)[train_index]

print np.unique(np.array(label_num)[train_index])

train_binary_proto = pd.get_dummies(train.iloc[train_index]['Proto'])
train_binary_state = pd.get_dummies(train.iloc[train_index]['State'])
train_binary_dir = pd.get_dummies(train.iloc[train_index]['Dir'])
train_binary_dport= pd.get_dummies(train.iloc[train_index]['Dport'])

train_copy_lda = pd.DataFrame()
train_copy_pca = pd.DataFrame()
#for column in column_str:
#    del train_copy_lda[column]
#del train_copy_lda['Label']

val_label = np.array(label_num)[train_index]
clf_lda = knn(n_neighbors=1, algorithm='kd_tree')
clf_pca = knn(n_neighbors=1, algorithm='kd_tree')
#clf_numeric = SVC(C=10,kernel='linear', class_weight='balanced')
#clf_lda = SVC(C=10,kernel='linear',class_weight='balanced')
for iteration in range(2):
    #print iteration
    #ten_fold = 0
    for train_val_index, test_val_index in skf:    
        #print ten_fold
        #ten_fold = ten_fold+1
        ''' 
        train_binary_proto = pd.get_dummies(train.iloc[train_index[train_val_index]]['Proto'])
        train_binary_state = pd.get_dummies(train.iloc[train_index[train_val_index]]['State'])
        train_binary_dir = pd.get_dummies(train.iloc[train_index[train_val_index]]['Dir'])
        train_binary_dport= pd.get_dummies(train.iloc[train_index[train_val_index]]['Dport'])
        test_binary_proto = np.zeros([len(test_val_index), len(train_binary_proto.columns)])
        test_binary_proto = np.zeros([len(test_val_index), len(train_binary_proto.columns)])
        test_binary_proto = np.zeros([len(test_val_index), len(train_binary_proto.columns)])
        test_binary_proto = np.zeros([len(test_val_index), len(train_binary_proto.columns)])

        (train_proto_lda, proto_lda) = lda_function(train_binary_proto, val_label[train_val_index])
        (train_state_lda, state_lda) = lda_function(train_binary_state, val_label[train_val_index])
        (train_dir_lda, dir_lda) = lda_function(train_binary_dir, val_label[train_val_index])
        (train_dport_lda, dport_lda) = lda_function(train_binary_dport, val_label[train_val_index])
        train_lda_np = np.append(train_proto_lda, train_state_lda, 1)
        train_lda_np = np.append(train_lda_np, train_dir_lda, 1)
        train_lda_np = np.append(train_lda_np, train_dport_lda, 1)

        '''
        #print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"

        (now_train_binary_proto,now_test_binary_proto) = del_zero_column(train_binary_proto.iloc[train_val_index],train_binary_proto.iloc[test_val_index])
        (now_train_binary_state,now_test_binary_state) = del_zero_column(train_binary_state.iloc[train_val_index], train_binary_state.iloc[test_val_index])
        (now_train_binary_dir,now_test_binary_dir) = del_zero_column(train_binary_dir.iloc[train_val_index], train_binary_dir.iloc[test_val_index])
        (now_train_binary_dport,now_test_binary_dport) = del_zero_column(train_binary_dport.iloc[train_val_index], train_binary_dport.iloc[test_val_index])
        (train_proto_lda, proto_lda) = lda_function(now_train_binary_proto, val_label[train_val_index])
        (train_state_lda, state_lda) = lda_function(now_train_binary_state, val_label[train_val_index])
        (train_dir_lda, dir_lda) = lda_function(now_train_binary_dir, val_label[train_val_index])
        (train_dport_lda, dport_lda) = lda_function(now_train_binary_dport, val_label[train_val_index])
        train_lda_np = np.append(train_proto_lda, train_state_lda, 1)
        train_lda_np = np.append(train_lda_np, train_dir_lda, 1)
        train_lda_np = np.append(train_lda_np, train_dport_lda, 1)


        test_proto_lda = proto_lda.transform(now_test_binary_proto)
        test_state_lda = state_lda.transform(now_test_binary_state)            
        test_dir_lda = dir_lda.transform(now_test_binary_dir)
        test_dport_lda = dport_lda.transform(now_test_binary_dport)
        test_lda_np = np.append(test_proto_lda, test_state_lda, 1)
        test_lda_np = np.append(test_lda_np, test_dir_lda, 1)
        test_lda_np = np.append(test_lda_np, test_dport_lda, 1)



        #train_numpy_pca = train_copy_pca.values
        #(train_numpy_pca_norm, test_numpy_pca_norm) = normalization(train_copy_pca.iloc[train_val_index],train_copy_pca.iloc[test_val_index], train_copy_pca.columns)
        (train_numpy_lda_norm, test_numpy_lda_norm) = normalization(train_lda_np,test_lda_np)

        #----Here--- continue
        clf_lda.fit(train_numpy_lda_norm, val_label[train_val_index])
        all_score_lda.append(f1_score(clf_lda.predict(test_numpy_lda_norm), val_label[test_val_index]))




    lda_fscore = np.average(all_score_lda)
    now_lda_fscore.append(lda_fscore)

output_fscore.loc[0] = [np.max(now_lda_fscore), np.min(now_lda_fscore), np.average(now_lda_fscore)]
#print 'numeric:', numeric_fscore, 'lda:', lda_fscore 



#print output_fscore.loc[str(index_train_dataset)]
output_fscore.to_csv('new_1_lda_3class.csv')
#output_fscore.loc[str(index_train_dataset)] = [numeric_fscore, lda_fscore]
#output_fscore.to_csv('4_string_numeric_vs_lda_no_norm.csv')

    
    

In [ ]:
key_str = {'Normal':0, 'Botnet':1, 'Background':2}
column_unuse = ['StartTime', 'SrcAddr', 'DstAddr', 'Sport']
column_str = ['Proto', 'State','Dir','Dport']
score_list = {}
fscore_list = {}
#output_fscore = pd.DataFrame()
output_fscore = pd.DataFrame(columns=['max', 'min','average'])
#output_fscore = pd.DataFrame(columns=['Numeric_proto', 'Numeric_state', 'Numeric_dir', 'Numeric_dport', 'Binary_proto', 'Binary_state', 'Binary_dir','Binary_dport'])

pca_rate = 0.9


for index_train_dataset in xrange(1,14): 
   
    train = pd.read_csv('botnet_'+str(index_train_dataset)+'.binetflow')

    # Fill 0 to missing value
    train = train.fillna(0)    
    train = train[train['State'] != 0]

    # Drop unuse columns and row with missing value
    missing_state = []
    map(lambda word: missing_state.append(word) if word[-1]=='_' else None, np.unique(train['State']))

    for column in column_unuse:
        del train[column]

    for state in missing_state:
        train = train[train['State'] != state]


    # Transfer str label to int lable
    label_num = []

    map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])
    train['Label'] = label_num



    
    # training & testing index
    train_index = np.array(np.where(train['Label'] == 0)[0].tolist() + np.where(train['Label'] == 1)[0].tolist())
    print 'Dataset:',index_train_dataset, 'training length:', len(train_index)
    
    now_lda_fscore = []
    
    all_score_lda = []
    
    skf = StratifiedKFold(np.array(label_num)[train_index], n_folds=5)
    val_label = np.array(label_num)[train_index]

    train_binary_proto = pd.get_dummies(train.iloc[train_index]['Proto'])
    train_binary_state = pd.get_dummies(train.iloc[train_index]['State'])
    train_binary_dir = pd.get_dummies(train.iloc[train_index]['Dir'])
    train_binary_dport= pd.get_dummies(train.iloc[train_index]['Dport'])

    train_copy_lda = pd.DataFrame()
    train_copy_pca = pd.DataFrame()
    #for column in column_str:
    #    del train_copy_lda[column]
    #del train_copy_lda['Label']
    
    val_label = np.array(label_num)[train_index]
    clf_lda = SVC(C=1, gamma=0.1, class_weight='balanced')
    clf_pca = knn(n_neighbors=1, algorithm='kd_tree')
    #clf_numeric = SVC(C=10,kernel='linear', class_weight='balanced')
    #clf_lda = SVC(C=10,kernel='linear',class_weight='balanced')
    for iteration in range(10):
        #print iteration
        #ten_fold = 0
        for train_val_index, test_val_index in skf:    
            #print ten_fold
            #ten_fold = ten_fold+1
            ''' 
            train_binary_proto = pd.get_dummies(train.iloc[train_index[train_val_index]]['Proto'])
            train_binary_state = pd.get_dummies(train.iloc[train_index[train_val_index]]['State'])
            train_binary_dir = pd.get_dummies(train.iloc[train_index[train_val_index]]['Dir'])
            train_binary_dport= pd.get_dummies(train.iloc[train_index[train_val_index]]['Dport'])
            test_binary_proto = np.zeros([len(test_val_index), len(train_binary_proto.columns)])
            test_binary_proto = np.zeros([len(test_val_index), len(train_binary_proto.columns)])
            test_binary_proto = np.zeros([len(test_val_index), len(train_binary_proto.columns)])
            test_binary_proto = np.zeros([len(test_val_index), len(train_binary_proto.columns)])

            (train_proto_lda, proto_lda) = lda_function(train_binary_proto, val_label[train_val_index])
            (train_state_lda, state_lda) = lda_function(train_binary_state, val_label[train_val_index])
            (train_dir_lda, dir_lda) = lda_function(train_binary_dir, val_label[train_val_index])
            (train_dport_lda, dport_lda) = lda_function(train_binary_dport, val_label[train_val_index])
            train_lda_np = np.append(train_proto_lda, train_state_lda, 1)
            train_lda_np = np.append(train_lda_np, train_dir_lda, 1)
            train_lda_np = np.append(train_lda_np, train_dport_lda, 1)
            
            '''
            #print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
            
            (now_train_binary_proto,now_test_binary_proto) = del_zero_column(train_binary_proto.iloc[train_val_index],train_binary_proto.iloc[test_val_index])
            (now_train_binary_state,now_test_binary_state) = del_zero_column(train_binary_state.iloc[train_val_index], train_binary_state.iloc[test_val_index])
            (now_train_binary_dir,now_test_binary_dir) = del_zero_column(train_binary_dir.iloc[train_val_index], train_binary_dir.iloc[test_val_index])
            (now_train_binary_dport,now_test_binary_dport) = del_zero_column(train_binary_dport.iloc[train_val_index], train_binary_dport.iloc[test_val_index])
            (train_proto_lda, proto_lda) = lda_function(now_train_binary_proto, val_label[train_val_index])
            (train_state_lda, state_lda) = lda_function(now_train_binary_state, val_label[train_val_index])
            (train_dir_lda, dir_lda) = lda_function(now_train_binary_dir, val_label[train_val_index])
            (train_dport_lda, dport_lda) = lda_function(now_train_binary_dport, val_label[train_val_index])
            train_lda_np = np.append(train_proto_lda, train_state_lda, 1)
            train_lda_np = np.append(train_lda_np, train_dir_lda, 1)
            train_lda_np = np.append(train_lda_np, train_dport_lda, 1)


            test_proto_lda = proto_lda.transform(now_test_binary_proto)
            test_state_lda = state_lda.transform(now_test_binary_state)            
            test_dir_lda = dir_lda.transform(now_test_binary_dir)
            test_dport_lda = dport_lda.transform(now_test_binary_dport)
            test_lda_np = np.append(test_proto_lda, test_state_lda, 1)
            test_lda_np = np.append(test_lda_np, test_dir_lda, 1)
            test_lda_np = np.append(test_lda_np, test_dport_lda, 1)
            
            

            #train_numpy_pca = train_copy_pca.values
            #(train_numpy_pca_norm, test_numpy_pca_norm) = normalization(train_copy_pca.iloc[train_val_index],train_copy_pca.iloc[test_val_index], train_copy_pca.columns)
            (train_numpy_lda_norm, test_numpy_lda_norm) = normalization(train_lda_np,test_lda_np)

            #----Here--- continue
            clf_lda.fit(train_numpy_lda_norm, val_label[train_val_index])
            all_score_lda.append(f1_score(clf_lda.predict(test_numpy_lda_norm), val_label[test_val_index]))

        
    
    
        lda_fscore = np.average(all_score_lda)
        now_lda_fscore.append(lda_fscore)
    print 'max:',np.max(now_lda_fscore),  'min:', np.min(now_lda_fscore),'avg:',np.average(now_lda_fscore)
    output_fscore.loc[str(index_train_dataset)] = [np.max(now_lda_fscore), np.min(now_lda_fscore), np.average(now_lda_fscore)]
    #print 'numeric:', numeric_fscore, 'lda:', lda_fscore 


    
    #print output_fscore.loc[str(index_train_dataset)]
    output_fscore.to_csv('lda_tine_5class(anomoly)_svc.csv')
    #output_fscore.loc[str(index_train_dataset)] = [numeric_fscore, lda_fscore]
    #output_fscore.to_csv('4_string_numeric_vs_lda_no_norm.csv')

    
    

# KDD

In [ ]:
attack_list = open('training_attack_types.txt')
attack_dic = {}

for line in attack_list.readlines():    
    detail = line.split(' ')
    if len(detail) == 1:
        break
    attack_type = detail[1][:-1]
    attack_name = detail[0]
    
    if attack_dic.has_key(attack_type):
        attack_dic[attack_type].append(attack_name)
    else:
        attack_dic[attack_type] = []
        attack_dic[attack_type].append(attack_name)

test_attack_dic = {}
test_attack_dic["apache2."] = 2 
test_attack_dic["back."] = 2 
test_attack_dic["buffer_overflow."] = 3 
test_attack_dic["ftp_write."] = 4 
test_attack_dic["guess_passwd."] = 4 
#test_attack_dic["httptunnel."] = 4 
test_attack_dic["httptunnel."] = 3 
test_attack_dic["imap."] = 4 
test_attack_dic["ipsweep."] = 1 
test_attack_dic["land."] = 2 
test_attack_dic["loadmodule."] = 3 
test_attack_dic["mailbomb."] = 2 
test_attack_dic["mscan."] = 1 
test_attack_dic["multihop."] = 4 
#test_attack_dic["multihop."] = 3 # note that this is a duplicate 
test_attack_dic["named."] = 4 
test_attack_dic["neptune."] = 2 
test_attack_dic["nmap."] = 1 
test_attack_dic["perl."] = 3 
test_attack_dic["phf."] = 4 
test_attack_dic["pod."] = 2 
test_attack_dic["portsweep."] = 1 
test_attack_dic["processtable."] = 2 
test_attack_dic["ps."] = 3 
test_attack_dic["rootkit."] = 3 
test_attack_dic["saint."] = 1 
test_attack_dic["satan."] = 1 
test_attack_dic["sendmail."] = 4 
test_attack_dic["smurf."] = 2 
test_attack_dic["snmpgetattack."] = 4 
test_attack_dic["snmpguess."] = 4 
test_attack_dic["sqlattack."] = 3 
test_attack_dic["teardrop."] = 2 
test_attack_dic["udpstorm."] = 2 
test_attack_dic["warezmaster."] = 4 
test_attack_dic["worm."] = 4 
test_attack_dic["xlock."] = 4 
test_attack_dic["xsnoop."] = 4 
test_attack_dic["xterm."] = 3
test_attack_dic['normal.'] = 0
attack_dic_num = {}
attack_dic_num[0] = ['normal']
attack_dic_num[2] = attack_dic['dos']
attack_dic_num[4] = attack_dic['r2l']
attack_dic_num[3] = attack_dic['u2r']
attack_dic_num[1] = attack_dic['probe']

In [ ]:
def del_zero_column(df_train, df_test):
    new_df_test = pd.DataFrame(np.zeros([df_test.shape[0], df_train.shape[1]]),columns=df_train.columns)
    new_df_test[list(set(df_train.columns) & set(df_test.columns))] = df_test[list(set(df_train.columns) & set(df_test.columns))]
    #df_test = df_test[df_train.columns]
    return (df_train, new_df_test)
def lda_function(X, y):
    n_cluster = 2
    
    original_y = y.copy()
    anomaly_index = []
    print "y:", np.unique(original_y)
    
    for uni_label in np.unique(y)[1:]:
        print uni_label       
        anomaly_index.append(np.where(y == uni_label)[0])

    km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
       
    for i in range(len(anomaly_index)):
        for (j,now_label) in zip(range(2*(i+1), 2*(i+1)-n_cluster, -1), range(n_cluster-1,0,-1)):
            print "i",i,"j:",j
            anomaly_label[i][np.where(anomaly_label == (now_label))[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)

In [ ]:
column_str = ['protocol_type', 'service','flag']
score_list = {}
fscore_list = {}

output_fscore = pd.DataFrame(columns=['max', 'min','average'])


train = pd.read_csv('kdd_10_per.csv', nrows=30000)
test= pd.read_csv('corrected', header=None, nrows=1000)
test.columns = train.columns

category_label_binary = []
map(lambda label: map(lambda cate: category_label_binary.append(cate) if label in attack_dic_num[cate] else None,attack_dic_num.keys()),train['label'])
category_label_binary = np.array(category_label_binary)
#category_label_binary[np.where((category_label_binary != 0))] = 1

print 'done'
test_category_label_binary = []
map(lambda label: test_category_label_binary.append(test_attack_dic[label]),test['label'])
test_category_label_binary = np.array(test_category_label_binary)
#test_category_label_binary[np.where((test_category_label_binary != 0))] = 1




print 'training length:', len(train)

now_lda_fscore = []

all_score_lda = []

skf = StratifiedKFold(train['label'], n_folds=10)
val_label = train['label']

train_binary_proto = pd.get_dummies(train['protocol_type'])
train_binary_state = pd.get_dummies(train['service'])
train_binary_flag = pd.get_dummies(train['flag'])
#train_binary_dport= pd.get_dummies(train.iloc[train_index]['Dport'])

test_binary_proto = pd.get_dummies(test['protocol_type'])
test_binary_state = pd.get_dummies(test['service'])
test_binary_flag = pd.get_dummies(test['flag'])

del train['label']
del test['label']
train_copy_lda = pd.DataFrame()
train_copy_pca = pd.DataFrame()
#for column in column_str:
#    del train_copy_lda[column]
#del train_copy_lda['Label']

#val_label = np.array(label_num)[train_index]
clf_lda = knn(n_neighbors=1, algorithm='kd_tree', n_jobs=-1)
clf_pca = knn(n_neighbors=1, algorithm='kd_tree')
#clf_numeric = SVC(C=10,kernel='linear', class_weight='balanced')
#clf_lda = SVC(C=10,kernel='linear',class_weight='balanced')

        
        #print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"

(now_train_binary_proto,now_test_binary_proto) = del_zero_column(train_binary_proto, test_binary_proto)
(now_train_binary_state,now_test_binary_state) = del_zero_column(train_binary_state, test_binary_state)
(now_train_binary_flag,now_test_binary_flag) = del_zero_column(train_binary_flag, test_binary_flag)
print "del column finish"
#(now_train_binary_dport,now_test_binary_dport) = del_zero_column(train_binary_dport.iloc[train_val_index], train_binary_dport.iloc[test_val_index])
#print np.unique(category_label_binary)
(train_proto_lda, proto_lda) = lda_function(now_train_binary_proto, category_label_binary)
(train_state_lda, state_lda) = lda_function(now_train_binary_state, category_label_binary)
(train_flag_lda, flag_lda) = lda_function(now_train_binary_flag, category_label_binary)
print "train lda finish"
#(train_dport_lda, dport_lda) = lda_function(now_train_binary_dport, val_label[train_val_index])
train_lda_np = np.append(train_proto_lda, train_state_lda, 1)
train_lda_np = np.append(train_lda_np, train_flag_lda, 1)
#train_lda_np = np.append(train_lda_np, train_dport_lda, 1)


test_proto_lda = proto_lda.transform(now_test_binary_proto)
test_state_lda = state_lda.transform(now_test_binary_state)            
test_flag_lda = flag_lda.transform(now_test_binary_dir)
print "test lda finish"
#test_dport_lda = dport_lda.transform(now_test_binary_dport)
test_lda_np = np.append(test_proto_lda, test_state_lda, 1)
test_lda_np = np.append(test_lda_np, test_flag_lda, 1)
#test_lda_np = np.append(test_lda_np, test_dport_lda, 1)



#train_numpy_pca = train_copy_pca.values
#(train_numpy_pca_norm, test_numpy_pca_norm) = normalization(train_copy_pca.iloc[train_val_index],train_copy_pca.iloc[test_val_index], train_copy_pca.columns)
(train_numpy_lda_norm, test_numpy_lda_norm) = normalization(train_lda_np,test_lda_np)

#----Here--- continue
clf_lda.fit(train_numpy_lda_norm, category_label_binary)

f_score = f1_score(clf_lda.predict(test_numpy_lda_norm), test_category_label_binary)

print f_score



   

    
    